In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv("main_database_large_diff.csv")

In [3]:
df.head(2)

,season,target,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
0,9,2,-15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,38.23,-3.330,-41.560
1,9,1,-10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.99,6.129,-5.861


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

def get_train_test_split(data):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(data, data["target"]):
        data_train = data.loc[train_index]
        data_test = data.loc[test_index]
    return data_train, data_test

train,test = get_train_test_split(df)

X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [5]:
X_train.head(2)

,season,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
1513,23,17.0,-1.22,0.72,-0.14,-1.28,-5.071,-2.393,-0.125,-1.667,-1.6,-1.285,-3.0,-2.8,-2.143,17.0,-68.11,-27.56,40.55
1165,20,11.0,-1.75,0.63,0.26,-1.26,-3.000,-1.625,-0.118,-2.333,-1.2,-1.285,-5.0,-2.2,-2.142,16.0,-63.21,-26.64,36.57


In [6]:
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[0]

array([ 0.25378566,  0.1875807 , -0.01346167,  0.00794459, -0.00154478,
       -0.01412372, -0.05595422, -0.02640474, -0.00137927, -0.01839394,
       -0.01765465, -0.01417889, -0.03310248, -0.03089565, -0.0236462 ,
        0.1875807 , -0.75153657, -0.30410142,  0.44743515])

In [7]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [8]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

class_weight = {0: 1.528,
                1: 2.111,
                2: 1}

In [9]:
model.fit(X_train_norm, y_train.values, epochs=20, validation_data=(X_test_norm,y_test), class_weight=class_weight)

Train on 1394 samples, validate on 349 samples
Epoch 1/20
1394/1394 [==============================] - 1s 396us/step - loss: 1.0894 - acc: 0.4039 - val_loss: 1.0660 - val_acc: 0.5330
Epoch 2/20
1394/1394 [==============================] - 0s 107us/step - loss: 1.0626 - acc: 0.5316 - val_loss: 1.0262 - val_acc: 0.5702
Epoch 3/20
1394/1394 [==============================] - 0s 81us/step - loss: 1.0352 - acc: 0.5552 - val_loss: 0.9874 - val_acc: 0.5817
Epoch 4/20
1394/1394 [==============================] - 0s 67us/step - loss: 1.0151 - acc: 0.5624 - val_loss: 0.9685 - val_acc: 0.5817
Epoch 5/20
1394/1394 [==============================] - ETA: 0s - loss: 1.0177 - acc: 0.559 - 0s 76us/step - loss: 1.0053 - acc: 0.5631 - val_loss: 0.9641 - val_acc: 0.5845
Epoch 6/20
1394/1394 [==============================] - 0s 75us/step - loss: 1.0005 - acc: 0.5624 - val_loss: 0.9630 - val_acc: 0.5817
Epoch 7/20
1394/1394 [==============================] - 0s 77us/step - loss: 0.9981 - acc: 0.5617 - val

In [10]:
train_pred = model.predict_classes(X_train_norm)
test_pred = model.predict_classes(X_test_norm)

import numpy as np

unique, counts = np.unique(train_pred, return_counts=True)
print("preds: " + str(dict(zip(unique, counts))))

unique, counts = np.unique(y_train.values, return_counts=True)
print("real: " + str(dict(zip(unique, counts))))

from sklearn.metrics import confusion_matrix as conf_matrix

print(conf_matrix(train_pred,y_train.values,labels=[0,1,2]))

preds: {0: 625, 1: 156, 2: 613}
real: {0: 429, 1: 310, 2: 655}
[[311 152 162]
 [ 43  53  60]
 [ 75 105 433]]


In [11]:
print(conf_matrix(test_pred,y_test.values))

[[ 85  40  48]
 [  9  10  10]
 [ 13  28 106]]


In [12]:
test_pred_probs = model.predict(X_test_norm)

In [13]:
test_pred_probs[0]

array([0.47439745, 0.29763237, 0.22797018], dtype=float32)

In [14]:
y_train.value_counts()

2    655
0    429
1    310
Name: target, dtype: int64

In [15]:
y_test.value_counts()

2    164
0    107
1     78
Name: target, dtype: int64

In [16]:
df.shape

(1743, 20)

In [17]:
# 2 = 655 + 164 = 700 + 110 + 9 = 819
# 1 = 310 + 78 = 388
# 0 = 429 + 107 = 536
819/388

2.1108247422680413

In [18]:
819/536

1.5279850746268657